# Install dependencies and preprocess data

## Setup & Installation

In [ ]:
# Step 1. Setup & Install Dependencies
!pip install -q tensorflow keras opencv-python matplotlib scikit-learn kagglehub tf-explain lime seaborn scikit-image

## Drive Mount & Directory Setup

In [ ]:
#ignore this part it's for access to my drive
import warnings
warnings.filterwarnings('ignore')

try:
    from google.colab import drive
    try:
        drive.flush_and_unmount()
    except:
        pass
    drive.mount('/content/drive', force_remount=True)
    DRIVE_AVAILABLE = True
    print("✅ Google Drive mounted successfully!")
except Exception as e:
    print(f"⚠️ Google Drive mounting failed: {e}")
    print("📁 Will save outputs locally to /content/ instead")
    DRIVE_AVAILABLE = False

if DRIVE_AVAILABLE:
    OUTPUT_DIR = '/content/drive/MyDrive/NeuroScan'
else:
    OUTPUT_DIR = '/content/NeuroScan'

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Environment setup complete!")
print(f" Output directory: {OUTPUT_DIR}")
print("="*60)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp "/content/drive/MyDrive/Best_Models_SIC/best_modelVGG16.keras" "/content/best_modelVGG16.keras"

## Import Libraries & Download Dataset

In [ ]:
# Dataset Download & Exploration (Phase 1)
import kagglehub
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# Download dataset
print("⬇️ Downloading dataset...")
try:
    path = kagglehub.dataset_download('shuvokumarbasakbd/brain-tumors-mri-crystal-clean-colorized-mri-data')
    print('✅ Dataset downloaded to:', path)
except Exception as e:
    print(f"❌ Dataset download failed: {e}")
    print("💡 Make sure you have internet connection and Kaggle credentials configured")
    raise

# Dataset exploration
train_path = os.path.join(path, 'dataset', 'train')
test_path = os.path.join(path, 'dataset', 'test')
validation_path = os.path.join(path, 'dataset', 'validation')

## Dataset Statistics

In [ ]:
print("\n" + "="*60)
print("📊 DATASET STATISTICS")
print("="*60)

#Count images per class
train_stats = {}
test_stats = {}
validation_stats = {}

for class_name in os.listdir(train_path):
    class_path = os.path.join(train_path, class_name)
    if os.path.isdir(class_path):
        count = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
        train_stats[class_name] = count

for class_name in os.listdir(test_path):
    class_path = os.path.join(test_path, class_name)
    if os.path.isdir(class_path):
        count = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
        test_stats[class_name] = count

if os.path.exists(validation_path):
    for class_name in os.listdir(validation_path):
        class_path = os.path.join(validation_path, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
            validation_stats[class_name] = count

print(f"\n📁 Training Set:")
for cls, count in sorted(train_stats.items()):
    print(f"  {cls}: {count:,} images")
print(f"  ─────────────────────")
print(f"  Total: {sum(train_stats.values()):,} images")

if validation_stats:
    print(f"\n📁 Validation Set:")
    for cls, count in sorted(validation_stats.items()):
        print(f"  {cls}: {count:,} images")
    print(f"  ─────────────────────")
    print(f"  Total: {sum(validation_stats.values()):,} images")

print(f"\n📁 Testing Set:")
for cls, count in sorted(test_stats.items()):
    print(f"  {cls}: {count:,} images")
print(f"  ─────────────────────")
print(f"  Total: {sum(test_stats.values()):,} images")

# Check class balance
print("\n📊 Class Distribution Analysis:")
total_train = sum(train_stats.values())
for cls, count in sorted(train_stats.items()):
    percentage = (count / total_train) * 100
    bar = '█' * int(percentage / 2)
    print(f"  {cls:20s}: {percentage:5.2f}% {bar}")


## Visualize Dataset Distribution

In [ ]:
# Visualize dataset distribution
num_plots = 3 if validation_stats else 2
fig, axes = plt.subplots(1, num_plots, figsize=(7*num_plots, 6))   # Increased height

if num_plots == 2:
    axes = [axes[0], axes[1]]

# Get classes and counts
classes = sorted(train_stats.keys())
train_counts = [train_stats[c] for c in classes]
test_counts = [test_stats[c] for c in classes]

# Dynamic offset function
def label_offset(values):
    return max(values) * 0.03

# --- TRAINING SET PLOT ---
axes[0].bar(classes, train_counts, color='skyblue', edgecolor='navy', linewidth=1.5)
axes[0].set_title('Training Set Distribution', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Number of Images', fontsize=11)
axes[0].set_xlabel('Class', fontsize=11)
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].tick_params(axis='x', rotation=45)

for i, v in enumerate(train_counts):
    axes[0].text(i, v + label_offset(train_counts), str(v),
                 ha='center', va='bottom', fontweight='bold')

# --- VALIDATION + TEST OR JUST TEST ---
if validation_stats:

    validation_counts = [validation_stats[c] for c in classes]

    # VALIDATION
    axes[1].bar(classes, validation_counts, color='lightgreen',
                edgecolor='darkgreen', linewidth=1.5)
    axes[1].set_title('Validation Set Distribution', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Number of Images', fontsize=11)
    axes[1].set_xlabel('Class', fontsize=11)
    axes[1].grid(axis='y', alpha=0.3, linestyle='--')
    axes[1].tick_params(axis='x', rotation=45)

    for i, v in enumerate(validation_counts):
        axes[1].text(i, v + label_offset(validation_counts), str(v),
                     ha='center', va='bottom', fontweight='bold')

    # TEST
    axes[2].bar(classes, test_counts, color='lightcoral',
                edgecolor='darkred', linewidth=1.5)
    axes[2].set_title('Testing Set Distribution', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Number of Images', fontsize=11)
    axes[2].set_xlabel('Class', fontsize=11)
    axes[2].grid(axis='y', alpha=0.3, linestyle='--')
    axes[2].tick_params(axis='x', rotation=45)

    for i, v in enumerate(test_counts):
        axes[2].text(i, v + label_offset(test_counts), str(v),
                     ha='center', va='bottom', fontweight='bold')

else:
    # TEST ONLY (no validation)
    axes[1].bar(classes, test_counts, color='lightcoral',
                edgecolor='darkred', linewidth=1.5)
    axes[1].set_title('Testing Set Distribution', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Number of Images', fontsize=11)
    axes[1].set_xlabel('Class', fontsize=11)
    axes[1].grid(axis='y', alpha=0.3, linestyle='--')
    axes[1].tick_params(axis='x', rotation=45)

    for i, v in enumerate(test_counts):
        axes[1].text(i, v + label_offset(test_counts), str(v),
                     ha='center', va='bottom', fontweight='bold')

# Adjust layout to prevent collisions
plt.subplots_adjust(top=0.88)
plt.tight_layout()

plt.savefig(f'{OUTPUT_DIR}/dataset_distribution.png',
            dpi=300, bbox_inches='tight')

print(f"\n💾 Saved: {OUTPUT_DIR}/dataset_distribution.png")
plt.show()



## Sample Images Visualization

In [ ]:
# Sample visualization
print("\n🖼️ Sample Images from Dataset:")
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

img_idx = 0
for class_name in sorted(train_stats.keys()):
    class_path = os.path.join(train_path, class_name)
    sample_imgs = [os.path.join(class_path, f) for f in os.listdir(class_path)[:2]
                   if f.endswith(('.jpg', '.png', '.jpeg'))]

    for img_path in sample_imgs:
        if img_idx < len(axes):
            img = plt.imread(img_path)
            axes[img_idx].imshow(img)
            axes[img_idx].set_title(f'{class_name}', fontsize=11, fontweight='bold')
            axes[img_idx].axis('off')
            img_idx += 1

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/sample_images.png', dpi=300, bbox_inches='tight')
print(f"💾 Saved: {OUTPUT_DIR}/sample_images.png")
plt.show()

print("\n✅ Phase 1 Complete: Dataset downloaded and explored")

## Import Preprocessing Libraries & Defining preprocessing Function

In [ ]:
# Step 3. Data Preprocessing (Phase 2)
import cv2
from skimage import exposure
from tqdm.auto import tqdm

IMG_SIZE = (224, 224)

print("\n" + "="*60)
print("🔬 DATA PREPROCESSING PIPELINE")
print("="*60)

def preprocess_mri_image(img_path):
    """
    MRI preprocessing:
    1. Load and convert to grayscale
    2. Skull stripping (brain extraction)
    3. Noise reduction
    4. Contrast enhancement (CLAHE)
    5. Normalization
    6. Resize to standard dimensions
    7. Convert to RGB (3 channels as per action plan)

    Returns: RGB image (224, 224, 3) normalized to [0, 1]
    """
    # Load image
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Could not load: {img_path}")

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Skull stripping using Otsu's thresholding
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, brain_mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    brain_mask = cv2.morphologyEx(brain_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    brain_mask = cv2.morphologyEx(brain_mask, cv2.MORPH_OPEN, kernel, iterations=1)

    # Apply mask
    brain_only = cv2.bitwise_and(gray, gray, mask=brain_mask)

    # Noise reduction
    denoised = cv2.fastNlMeansDenoising(brain_only, None, h=10,
                                        templateWindowSize=7, searchWindowSize=21)

    # CLAHE for contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(denoised)

    # Normalization (Z-score on non-zero pixels)
    non_zero = enhanced[enhanced > 0]
    if len(non_zero) > 0:
        mean_val = np.mean(non_zero)
        std_val = np.std(non_zero)
        if std_val > 0:
            normalized = np.where(enhanced > 0, (enhanced - mean_val) / std_val, 0)
            normalized = (normalized - normalized.min()) / (normalized.max() - normalized.min() + 1e-8)
        else:
            normalized = enhanced / 255.0
    else:
        normalized = enhanced / 255.0

    # Resize to target dimensions
    resized = cv2.resize(normalized, IMG_SIZE, interpolation=cv2.INTER_AREA)

    # Convert to RGB (3 channels like mentioned in the action plan)
    rgb = cv2.cvtColor((resized * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB)
    rgb = rgb.astype(np.float32) / 255.0

    # Convert RGB to BGR before returning
    bgr = cv2.cvtColor((rgb * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
    bgr = bgr.astype(np.float32) / 255.0

    return bgr  # Returns BGR format (what cv2.imwrite expects)


## Preprocessing Visualization Function

In [ ]:
# Visualize preprocessing steps
def show_preprocessing_steps(img_path):
    """Visualize each preprocessing step"""
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    brain_only = cv2.bitwise_and(gray, gray, mask=mask)
    denoised = cv2.fastNlMeansDenoising(brain_only, None, h=10, templateWindowSize=7, searchWindowSize=21)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(denoised)
    final = preprocess_mri_image(img_path)

    fig, axes = plt.subplots(2, 4, figsize=(18, 9))

    axes[0, 0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('1. Original', fontweight='bold', fontsize=12)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(gray, cmap='gray')
    axes[0, 1].set_title('2. Grayscale', fontweight='bold', fontsize=12)
    axes[0, 1].axis('off')

    axes[0, 2].imshow(mask, cmap='gray')
    axes[0, 2].set_title('3. Brain Mask', fontweight='bold', fontsize=12)
    axes[0, 2].axis('off')

    axes[0, 3].imshow(brain_only, cmap='gray')
    axes[0, 3].set_title('4. Brain Extracted', fontweight='bold', fontsize=12)
    axes[0, 3].axis('off')

    axes[1, 0].imshow(denoised, cmap='gray')
    axes[1, 0].set_title('5. Denoised', fontweight='bold', fontsize=12)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(enhanced, cmap='gray')
    axes[1, 1].set_title('6. CLAHE Enhanced', fontweight='bold', fontsize=12)
    axes[1, 1].axis('off')

    axes[1, 2].imshow(cv2.cvtColor((final * 255).astype(np.uint8), cv2.COLOR_BGR2RGB))
    axes[1, 2].set_title('7. Final Preprocessed (RGB)', fontweight='bold', fontsize=12)
    axes[1, 2].axis('off')

    axes[1, 3].hist(gray.ravel(), bins=50, alpha=0.6, label='Original', color='blue', edgecolor='navy')
    axes[1, 3].hist(enhanced.ravel(), bins=50, alpha=0.6, label='Enhanced', color='red', edgecolor='darkred')
    axes[1, 3].set_title('8. Histogram Comparison', fontweight='bold', fontsize=12)
    axes[1, 3].set_xlabel('Pixel Intensity')
    axes[1, 3].set_ylabel('Frequency')
    axes[1, 3].legend()
    axes[1, 3].grid(alpha=0.3)

    plt.tight_layout()
    return fig


## Show Sample Preprocessing

In [ ]:
# Show preprocessing on samples from each class
print("\n🔍 Visualizing preprocessing pipeline...")
for class_name in sorted(train_stats.keys())[:2]:
    class_path = os.path.join(train_path, class_name)
    sample_file = [os.path.join(class_path, f) for f in os.listdir(class_path)
                   if f.endswith(('.jpg', '.png', '.jpeg'))][0]

    print(f"\n📸 Processing sample from: {class_name}")
    fig = show_preprocessing_steps(sample_file)
    save_path = f'{OUTPUT_DIR}/preprocessing_{class_name}.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"💾 Saved: {save_path}")
    plt.show()

## Dataset Preprocessing Function

In [ ]:
# Preprocess entire dataset
def preprocess_dataset(input_dir, output_dir):
    """Preprocess and save entire dataset"""
    os.makedirs(output_dir, exist_ok=True)

    total_processed = 0
    total_errors = 0

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        output_class = os.path.join(output_dir, class_name)
        os.makedirs(output_class, exist_ok=True)

        files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

        print(f"\n🔄 Processing {class_name}: {len(files)} images...")
        for img_file in tqdm(files, desc=class_name):
            try:
                img_path = os.path.join(class_path, img_file)
                preprocessed = preprocess_mri_image(img_path)  # Returns BGR
                output_path = os.path.join(output_class, img_file)
                cv2.imwrite(output_path, (preprocessed * 255).astype(np.uint8))
                total_processed += 1
            except Exception as e:
                print(f"\n⚠️ Error processing {img_file}: {e}")
                total_errors += 1

    return total_processed, total_errors


## Running Full Dataset Preprocessing

In [ ]:
print("\n" + "="*60)
print("🚀 STARTING FULL DATASET PREPROCESSING")
print("="*60)

# Save to writable directory instead of read-only dataset path
prep_train = os.path.join(OUTPUT_DIR, 'Preprocessed_Training')
prep_test = os.path.join(OUTPUT_DIR, 'Preprocessed_Testing')
prep_validation = os.path.join(OUTPUT_DIR, 'Preprocessed_Validation')

print("\n📂 Preprocessing Training Set...")
train_processed, train_errors = preprocess_dataset(train_path, prep_train)

# Preprocess Validation Set
if os.path.exists(validation_path) and validation_stats:
    print("\n📂 Preprocessing Validation Set...")
    validation_processed, validation_errors = preprocess_dataset(validation_path, prep_validation)
else:
    validation_processed, validation_errors = 0, 0
    print("\n⚠️ Validation folder not found. Skipping validation preprocessing...")

print("\n📂 Preprocessing Testing Set...")
test_processed, test_errors = preprocess_dataset(test_path, prep_test)

## Summary & Report Generation

In [ ]:
print("\n" + "="*60)
print("📊 PREPROCESSING SUMMARY")
print("="*60)
print(f"✅ Training images processed: {train_processed:,}")
print(f"❌ Training errors: {train_errors}")
if validation_processed > 0:
    print(f"✅ Validation images processed: {validation_processed:,}")
    print(f"❌ Validation errors: {validation_errors}")
print(f"✅ Testing images processed: {test_processed:,}")
print(f"❌ Testing errors: {test_errors}")
print(f"📁 Output location: {path}")

# Save summary report
if validation_processed > 0:
    summary_text = f"""
NeuroScan Preprocessing Report
{'='*60}
Generated: {pd.Timestamp.now()}

Dataset Statistics:
- Training images: {sum(train_stats.values()):,}
- Validation images: {sum(validation_stats.values()):,}
- Testing images: {sum(test_stats.values()):,}
- Classes: {len(train_stats)}

Preprocessing Results:
- Training processed: {train_processed:,}
- Training errors: {train_errors}
- Validation processed: {validation_processed:,}
- Validation errors: {validation_errors}
- Testing processed: {test_processed:,}
- Testing errors: {test_errors}

Output Directories:
- Preprocessed Training: {prep_train}
- Preprocessed Validation: {prep_validation}
- Preprocessed Testing: {prep_test}
- Visualizations: {OUTPUT_DIR}
"""
else:
    summary_text = f"""
NeuroScan Preprocessing Report
{'='*60}
Generated: {pd.Timestamp.now()}

Dataset Statistics:
- Training images: {sum(train_stats.values()):,}
- Testing images: {sum(test_stats.values()):,}
- Classes: {len(train_stats)}

Preprocessing Results:
- Training processed: {train_processed:,}
- Training errors: {train_errors}
- Testing processed: {test_processed:,}
- Testing errors: {test_errors}

Output Directories:
- Preprocessed Training: {prep_train}
- Preprocessed Testing: {prep_test}
- Visualizations: {OUTPUT_DIR}
"""

with open(f'{OUTPUT_DIR}/preprocessing_report.txt', 'w') as f:
    f.write(summary_text)

print(f"\n📄 Report saved: {OUTPUT_DIR}/preprocessing_report.txt")
print(f"\n✅ Phase 2 Complete: Data preprocessing finished" + (" (including validation)" if validation_processed > 0 else " (train + test only)"))

In [ ]:
# Move dataset
# !mv /root/.cache/kagglehub/datasets/shuvokumarbasakbd/brain-tumors-mri-crystal-clean-colorized-mri-data/versions/1/ /content/NeuroScan/brain-tumors-mri

# Data augmentation and Automated Data loading for training

### Data Augmentation Configuration

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.utils.class_weight import compute_class_weight

print("\n" + "="*60)
print("DATA AUGMENTATION & AUTOMATED DATA LOADING")
print("="*60)
print("\nCONFIGURING DATA AUGMENTATION")
print("="*60)

# Use preprocessed data paths
train_path = prep_train
validation_path = prep_validation
test_path = prep_test

print("Using preprocessed dataset paths:")
print("Training:", train_path)
print("Testing:", test_path)

# Configure data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    # width_shift_range=0.15,
    # height_shift_range=0.15,
    # shear_range=0.15,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    # fill_mode='nearest',
    brightness_range=[0.8, 1.2],
    # validation_split=0.15
)

val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    # validation_split=0.15
)

print("Data augmentation configured")


## Automated data loading pipeline

In [ ]:
print("\nSETTING UP AUTOMATED DATA LOADING")
print("="*60)

BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

print("Creating data generators...")

# Create data generators with sparse labels for compatibility
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    # color_mode='grayscale',
    # subset='training',
    shuffle=True,
    seed=42
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_path,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    # subset='validation',
    # color_mode='grayscale',
    shuffle=False,
    seed=42
)

test_generator = val_test_datagen.flow_from_directory(
    test_path,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    # color_mode='grayscale',
    shuffle=False,
    seed=42
)

# Use more significant names
train_dataset = train_generator
val_dataset = validation_generator
test_dataset = test_generator

print("Data generators created successfully")

## Class Balancing & Weights

In [ ]:
print("\nANALYZING CLASS DISTRIBUTION")
print("="*60)

class_counts = train_generator.classes
unique, counts = np.unique(class_counts, return_counts=True)
class_names = list(train_generator.class_indices.keys())

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_counts),
    y=class_counts
)

class_weights_dict = dict(zip(unique, class_weights))

print("Class Distribution:")
total_samples = sum(counts)
for class_idx, class_name in enumerate(class_names):
    count = counts[class_idx]
    percentage = (count / total_samples) * 100
    weight = class_weights_dict[class_idx]
    print(f"{class_name}: {count} images ({percentage:.1f}%) | Weight: {weight:.3f}")

print("Dataset is perfectly balanced")

## Augmentation Visualization





In [ ]:
print("\nVISUALIZING DATA AUGMENTATION")
print("="*60)

def visualize_augmentations(generator, num_samples=8):
    images, labels = next(generator)

    fig, axes = plt.subplots(2, 4, figsize=(15, 8))
    axes = axes.flatten()

    for i in range(min(num_samples, len(images))):
        img = images[i]
        img = np.clip(img, 0, 1)

        axes[i].imshow(img)
        axes[i].set_title(f'Class: {class_names[int(labels[i])]}')
        axes[i].axis('off')

    plt.suptitle('Data Augmentation Examples', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_augmentations(train_generator)
print("Augmentation visualization completed")


## Visualization & Statistics

In [ ]:
print("\n" + "="*60)
print("PIPELINE STATISTICS & SUMMARY")
print("="*60)

print("Pipeline Metrics:")
print(f"Training Samples: {train_generator.samples}")
print(f"Validation Samples: {validation_generator.samples}")
print(f"Test Samples: {test_generator.samples}")
print(f"Number of Classes: {len(class_names)}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Image Size: {TARGET_SIZE}")

print("\nTask Completed:")
print("Data augmentation and automated loading pipeline ready")

train_dataset = train_generator
val_dataset = validation_generator
test_dataset = test_generator

print("Using data generators from augmentation pipeline")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")



# CNN model architecture

## Download and import libraries

In [ ]:
!pip install -q git+https://github.com/tensorflow/addons.git
import tensorflow as tf
import tensorflow_addons as tfa

print("TensorFlow version:", tf.__version__)
print("TensorFlow Addons version:", tfa.__version__)

import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
import random

import os
import cv2
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

## Base model from scratch

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')    # ← 4 classes + softmax
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',   # ← changed
    metrics=['accuracy']
)

model.summary()

### Callbacks definition and training

In [ ]:
# Callbacks
callbacks = [
    # EarlyStopping : arrêter l'entraînement quand la performance ne s'améliore plus
    EarlyStopping(
        monitor='val_loss',           # Surveiller la perte de validation
        patience=5,                   # Nombre d'époques sans amélioration avant arrêt
        restore_best_weights=True,    # Restaurer les poids du meilleur modèle
        verbose=1
    ),

    # ModelCheckpoint : sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath='best_model_base.keras',  # Nom du fichier de sauvegarde
        monitor='val_loss',                # Surveiller la précision de validation
        save_best_only=True,               # Sauvegarder seulement le meilleur modèle
        mode='min',                        # Mode 'max' car on veut maximiser l'accuracy
        verbose=1
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-7, verbose=1)

]

# Entraîner le modèle
history = model.fit(
    train_dataset,                    # Dataset d'entraînement
    epochs=50,                        # Maximum d'époques
    validation_data=val_dataset,      # Dataset de validation
    callbacks=callbacks,              # Callbacks définis ci-dessus
    verbose=1
)

print("Entraînement terminé!")
print(f"Meilleur modèle sauvegardé sous: best_model_base.keras")

### Training graphs

In [ ]:
import matplotlib.pyplot as plt

# Afficher les courbes de loss et d'accuracy
plt.figure(figsize=(12, 4))

# Courbe de loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Courbe d'accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Model metrics

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import load_model

model = load_model('/content/best_model_base.keras')

# Predict
y_pred = model.predict(test_dataset, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# True labels (correct order)
y_true = test_dataset.labels

# Class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

print("=" * 70)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(y_true, y_pred_classes,
                            target_names=class_names,
                            digits=4))

print("=" * 70)
print("GLOBAL METRICS")
print("=" * 70)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

### Confusion matrix

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Define class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

# Visualization with Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": 12, "weight": "bold"})

plt.title('Confusion Matrix - Brain Tumor Classification',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Labels', fontsize=14, fontweight='bold')
plt.ylabel('True Labels', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## Transfer learning

### VGG19 with imagenet


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Charger un modèle pré-entraîné (VGG19)
base_model = VGG19(include_top=False, input_shape=(224,224,3), weights='imagenet')

# Dégeler partiellement le modèle (ex : les 10 dernières couches)
for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Ajouter des couches personnalisées
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)  # Changé à 4 pour vos 4 classes

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
# model.summary()


In [ ]:
# Callbacks
callbacks = [
    # EarlyStopping : arrêter l'entraînement quand la performance ne s'améliore plus
    EarlyStopping(
        monitor='val_loss',           # Surveiller la perte de validation
        patience=3,                   # Nombre d'époques sans amélioration avant arrêt
        restore_best_weights=True,    # Restaurer les poids du meilleur modèle
        verbose=1
    ),

    # ModelCheckpoint : sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath='best_modelVGG19.keras',  # Nom du fichier de sauvegarde
        monitor='val_accuracy',            # Surveiller la précision de validation
        save_best_only=True,               # Sauvegarder seulement le meilleur modèle
        mode='max',                        # Mode 'max' car on veut maximiser l'accuracy
        verbose=1
    )
]

# Entraîner le modèle
history = model.fit(
    train_dataset,                    # Dataset d'entraînement
    epochs=50,                        # Maximum d'époques
    validation_data=val_dataset,      # Dataset de validation
    callbacks=callbacks,              # Callbacks définis ci-dessus
    verbose=1
)

print("Entraînement terminé!")
print(f"Meilleur modèle sauvegardé sous: best_modelVGG19.keras")

In [ ]:
#Écrivez le code permettant de recharger un modèle Keras déjà entraîné et sauvegardé.
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

model = load_model('best_modelVGG19.keras')

# Afficher les courbes de loss et d'accuracy
plt.figure(figsize=(12, 4))

# Courbe de loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Courbe d'accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Predict
y_pred = model.predict(test_dataset, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# True labels (correct order)
y_true = test_dataset.labels

# Class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

print("=" * 70)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(y_true, y_pred_classes,
                            target_names=class_names,
                            digits=4))

print("=" * 70)
print("GLOBAL METRICS")
print("=" * 70)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Define class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

# Visualization with Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": 12, "weight": "bold"})

plt.title('Confusion Matrix - Brain Tumor Classification',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Labels', fontsize=14, fontweight='bold')
plt.ylabel('True Labels', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Charger un modèle pré-entraîné (VGG19)
base_model = VGG16(include_top=False, input_shape=(224,224,3), weights='imagenet')

# Dégeler partiellement le modèle (ex : les 10 dernières couches)
for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Ajouter des couches personnalisées
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)  # Changé à 4 pour vos 4 classes

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
# model.summary()

In [ ]:
# Callbacks
callbacks = [
    # EarlyStopping : arrêter l'entraînement quand la performance ne s'améliore plus
    EarlyStopping(
        monitor='val_loss',           # Surveiller la perte de validation
        patience=3,                   # Nombre d'époques sans amélioration avant arrêt
        restore_best_weights=True,    # Restaurer les poids du meilleur modèle
        verbose=1
    ),

    # ModelCheckpoint : sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath='best_modelVGG16.keras',  # Nom du fichier de sauvegarde
        monitor='val_accuracy',            # Surveiller la précision de validation
        save_best_only=True,               # Sauvegarder seulement le meilleur modèle
        mode='max',                        # Mode 'max' car on veut maximiser l'accuracy
        verbose=1
    )
]

# Entraîner le modèle
history = model.fit(
    train_dataset,                    # Dataset d'entraînement
    epochs=50,                        # Maximum d'époques
    validation_data=val_dataset,      # Dataset de validation
    callbacks=callbacks,              # Callbacks définis ci-dessus
    verbose=1
)

print("Entraînement terminé!")
print(f"Meilleur modèle sauvegardé sous: best_modelVGG16.keras")

In [ ]:
#Écrivez le code permettant de recharger un modèle Keras déjà entraîné et sauvegardé.
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

model = load_model('best_modelVGG16.keras')

# Afficher les courbes de loss et d'accuracy
plt.figure(figsize=(12, 4))

# Courbe de loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Courbe d'accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Predict
y_pred = model.predict(test_dataset, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# True labels (correct order)
y_true = test_dataset.labels

# Class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

print("=" * 70)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(y_true, y_pred_classes,
                            target_names=class_names,
                            digits=4))

print("=" * 70)
print("GLOBAL METRICS")
print("=" * 70)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Define class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

# Visualization with Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": 12, "weight": "bold"})

plt.title('Confusion Matrix - Brain Tumor Classification',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Labels', fontsize=14, fontweight='bold')
plt.ylabel('True Labels', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### MobileNet V2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Charger un modèle pré-entraîné (ResNet50)
base_model = MobileNetV2(include_top=False, input_shape=(224,224,3), weights='imagenet')

# Dégeler partiellement le modèle (ex : les 20 dernières couches)
for layer in base_model.layers[:-20]:
    layer.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Ajouter des couches personnalisées
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(4, activation='softmax')(x)

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
# model.summary()

In [ ]:
# Callbacks
callbacks = [
    # EarlyStopping : arrêter l'entraînement quand la performance ne s'améliore plus
    EarlyStopping(
        monitor='val_loss',           # Surveiller la perte de validation
        patience=5,                   # Nombre d'époques sans amélioration avant arrêt
        restore_best_weights=True,    # Restaurer les poids du meilleur modèle
        verbose=1
    ),

    # ModelCheckpoint : sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath='best_modelMobileNetV2.keras',  # Nom du fichier de sauvegarde
        monitor='val_accuracy',               # Surveiller la précision de validation
        save_best_only=True,                  # Sauvegarder seulement le meilleur modèle
        mode='max',                           # Mode 'max' car on veut maximiser l'accuracy
        verbose=1
    )
]
# Entraîner le modèle
history = model.fit(
    train_dataset,                    # Dataset d'entraînement
    epochs=50,                        # Maximum d'époques
    validation_data=val_dataset,      # Dataset de validation
    callbacks=callbacks,              # Callbacks définis ci-dessus
    verbose=1
)

print("Entraînement terminé!")
print(f"Meilleur modèle sauvegardé sous: best_modelMobileNetV2.keras")

In [ ]:
#Écrivez le code permettant de recharger un modèle Keras déjà entraîné et sauvegardé.
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

model = load_model('best_modelMobileNetV2.keras')

# Afficher les courbes de loss et d'accuracy
plt.figure(figsize=(12, 4))

# Courbe de loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Courbe d'accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Predict
y_pred = model.predict(test_dataset, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# True labels (correct order)
y_true = test_dataset.labels

# Class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

print("=" * 70)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(y_true, y_pred_classes,
                            target_names=class_names,
                            digits=4))

print("=" * 70)
print("GLOBAL METRICS")
print("=" * 70)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Define class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

# Visualization with Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": 12, "weight": "bold"})

plt.title('Confusion Matrix - Brain Tumor Classification',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Labels', fontsize=14, fontweight='bold')
plt.ylabel('True Labels', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### ResNet50 V2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Charger un modèle pré-entraîné (ResNet50)
base_model = ResNet50V2(include_top=False, input_shape=(224,224,3), weights='imagenet')

# Dégeler partiellement le modèle (ex : les 10 dernières couches)
for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Ajouter des couches personnalisées
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(4, activation='softmax')(x)

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
# model.summary()

In [ ]:
# Callbacks
callbacks = [
    # EarlyStopping : arrêter l'entraînement quand la performance ne s'améliore plus
    EarlyStopping(
        monitor='val_loss',           # Surveiller la perte de validation
        patience=5,                   # Nombre d'époques sans amélioration avant arrêt
        restore_best_weights=True,    # Restaurer les poids du meilleur modèle
        verbose=1
    ),

    # ModelCheckpoint : sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath='best_modelResNet50V2.keras',  # Nom du fichier de sauvegarde
        monitor='val_accuracy',               # Surveiller la précision de validation
        save_best_only=True,                  # Sauvegarder seulement le meilleur modèle
        mode='max',                           # Mode 'max' car on veut maximiser l'accuracy
        verbose=1
    )
]
# Entraîner le modèle
history = model.fit(
    train_dataset,                    # Dataset d'entraînement
    epochs=50,                        # Maximum d'époques
    validation_data=val_dataset,      # Dataset de validation
    callbacks=callbacks,              # Callbacks définis ci-dessus
    verbose=1
)

print("Entraînement terminé!")
print(f"Meilleur modèle sauvegardé sous: best_modelResNet50V2.keras")

In [ ]:
#Écrivez le code permettant de recharger un modèle Keras déjà entraîné et sauvegardé.
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

model = load_model('best_modelResNet50V2.keras')

# Afficher les courbes de loss et d'accuracy
plt.figure(figsize=(12, 4))

# Courbe de loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Courbe d'accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Predict
y_pred = model.predict(test_dataset, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# True labels (correct order)
y_true = test_dataset.labels

# Class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

print("=" * 70)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(y_true, y_pred_classes,
                            target_names=class_names,
                            digits=4))

print("=" * 70)
print("GLOBAL METRICS")
print("=" * 70)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Define class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

# Visualization with Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": 12, "weight": "bold"})

plt.title('Confusion Matrix - Brain Tumor Classification',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Labels', fontsize=14, fontweight='bold')
plt.ylabel('True Labels', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### Resnet 152 V2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Charger un modèle pré-entraîné (ResNet50)
base_model = ResNet152V2(include_top=False, input_shape=(224,224,3), weights='imagenet')

# Dégeler partiellement le modèle (ex : les 10 dernières couches)
for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Ajouter des couches personnalisées
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(4, activation='softmax')(x)

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
# model.summary()

In [ ]:
# Callbacks
callbacks = [
    # EarlyStopping : arrêter l'entraînement quand la performance ne s'améliore plus
    EarlyStopping(
        monitor='val_loss',           # Surveiller la perte de validation
        patience=3,                   # Nombre d'époques sans amélioration avant arrêt
        restore_best_weights=True,    # Restaurer les poids du meilleur modèle
        verbose=1
    ),

    # ModelCheckpoint : sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath='best_modelResNet152V2.keras',  # Nom du fichier de sauvegarde
        monitor='val_accuracy',               # Surveiller la précision de validation
        save_best_only=True,                  # Sauvegarder seulement le meilleur modèle
        mode='max',                           # Mode 'max' car on veut maximiser l'accuracy
        verbose=1
    )
]
# Entraîner le modèle
history = model.fit(
    train_dataset,                    # Dataset d'entraînement
    epochs=50,                        # Maximum d'époques
    validation_data=val_dataset,      # Dataset de validation
    callbacks=callbacks,              # Callbacks définis ci-dessus
    verbose=1
)

print("Entraînement terminé!")
print(f"Meilleur modèle sauvegardé sous: best_modelResNet152V2.keras")

In [ ]:
#Écrivez le code permettant de recharger un modèle Keras déjà entraîné et sauvegardé.
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

model = load_model('best_modelResNet152V2.keras')

# Afficher les courbes de loss et d'accuracy
plt.figure(figsize=(12, 4))

# Courbe de loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Courbe d'accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Predict
y_pred = model.predict(test_dataset, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# True labels (correct order)
y_true = test_dataset.labels

# Class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

print("=" * 70)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(y_true, y_pred_classes,
                            target_names=class_names,
                            digits=4))

print("=" * 70)
print("GLOBAL METRICS")
print("=" * 70)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Define class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

# Visualization with Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": 12, "weight": "bold"})

plt.title('Confusion Matrix - Brain Tumor Classification',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Labels', fontsize=14, fontweight='bold')
plt.ylabel('True Labels', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### EfficientNetV2 B0

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input  # IMPORTANT !
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Charger EfficientNetB0
base_model = EfficientNetV2B0(
    include_top=False,
    input_shape=(224, 224, 3),
    weights='imagenet',
    include_preprocessing=True
)

# Dégeler davantage de couches (10 dernières)
for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Ajouter des couches de classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)  # Plus de neurones
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(4, activation='softmax')(x)

# Créer le modèle
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler avec learning rate plus élevé
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Callbacks
callbacks = [
    # EarlyStopping : arrêter l'entraînement quand la performance ne s'améliore plus
    EarlyStopping(
        monitor='val_loss',           # Surveiller la perte de validation
        patience=5,                   # Nombre d'époques sans amélioration avant arrêt
        restore_best_weights=True,    # Restaurer les poids du meilleur modèle
        verbose=1
    ),

    # ModelCheckpoint : sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath='best_EfficientNetV2B0.keras',  # Nom du fichier de sauvegarde
        monitor='val_accuracy',               # Surveiller la précision de validation
        save_best_only=True,                  # Sauvegarder seulement le meilleur modèle
        mode='max',                           # Mode 'max' car on veut maximiser l'accuracy
        verbose=1
    ),
    ReduceLROnPlateau(factor=0.5, patience=5)
]
# Entraîner le modèle
history = model.fit(
    train_dataset,                    # Dataset d'entraînement
    epochs=50,                        # Maximum d'époques
    validation_data=val_dataset,      # Dataset de validation
    callbacks=callbacks,              # Callbacks définis ci-dessus
    verbose=1
)

print("Entraînement terminé!")
print(f"Meilleur modèle sauvegardé sous: best_EfficientNetV2B0.keras")

In [ ]:
#Écrivez le code permettant de recharger un modèle Keras déjà entraîné et sauvegardé.
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

model = load_model('best_EfficientNetV2B0.keras')

# Afficher les courbes de loss et d'accuracy
plt.figure(figsize=(12, 4))

# Courbe de loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Courbe d'accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Predict
y_pred = model.predict(test_dataset, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# True labels (correct order)
y_true = test_dataset.labels

# Class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

print("=" * 70)
print("DETAILED CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(y_true, y_pred_classes,
                            target_names=class_names,
                            digits=4))

print("=" * 70)
print("GLOBAL METRICS")
print("=" * 70)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Define class names
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

# Visualization with Seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": 12, "weight": "bold"})

plt.title('Confusion Matrix - Brain Tumor Classification',
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Labels', fontsize=14, fontweight='bold')
plt.ylabel('True Labels', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


# XAI for VGG16

## Grad-cam

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load model
model = tf.keras.models.load_model("/content/best_modelVGG16.keras")
IMG_SIZE = 224

# ----------------------------
# 1. Preprocessing
# ----------------------------
def preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# ----------------------------
# 2. FIXED Grad-CAM
# ----------------------------
def make_gradcam_heatmap(model, img_array, last_conv_layer_name="block5_conv3"):

    # Create Grad-CAM model
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[
            model.get_layer(last_conv_layer_name).output,
            model.output
        ]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    # Compute gradients
    grads = tape.gradient(loss, conv_outputs)

    # Compute channel weights
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply weights × feature maps
    conv_outputs = conv_outputs[0]
    heatmap = tf.zeros(conv_outputs.shape[:2], dtype=tf.float32)

    for i in range(pooled_grads.shape[0]):
        heatmap += pooled_grads[i] * conv_outputs[:, :, i]

    # Normalize the heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    return heatmap.numpy()


# ----------------------------
# 3. Display Grad-CAM correctly
# ----------------------------
def display_gradcam(img_path):

    # Preprocess for prediction
    img_array = preprocess_image(img_path)

    # Create heatmap
    heatmap = make_gradcam_heatmap(model, img_array)

    # Read original image in RGB
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

    # Resize heatmap to image size
    heatmap = cv2.resize(heatmap, (IMG_SIZE, IMG_SIZE))

    # Convert heatmap to color
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Superimpose
    superimposed = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

    # Display
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title("Original")

    plt.subplot(1,2,2)
    plt.imshow(superimposed)
    plt.title("Grad-CAM")

    plt.show()




In [ ]:
# Example
display_gradcam("/content/NeuroScan/Preprocessed_Training/meningioma_tumor/M_101_DA__7.jpg")

## Grad-cam++

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2

def display_gradcam_plus_plus(model, img_path, last_conv_layer_name="block5_conv3", class_names=None):
    """
    Generates Grad-CAM++ overlay for a single image and displays side by side with the original.

    Parameters:
    - model: loaded Keras model
    - img_path: path to the image
    - last_conv_layer_name: name of the last conv layer in the model
    - class_names: optional list of class names to display
    """

    # --- Load and preprocess image ---
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_batch)

    # --- Prediction ---
    preds = model.predict(img_preprocessed)
    predicted_class = np.argmax(preds)
    confidence = np.max(preds)
    class_label = class_names[predicted_class] if class_names else str(predicted_class)

    print(f"✅ Predicted class: {class_label} | Confidence: {confidence:.4f}")

    # --- Grad-CAM++ function ---
    def gradcam_plus_plus(model, img_array, layer_name, class_index=None):
        conv_layer = model.get_layer(layer_name)
        grad_model = tf.keras.Model(
            inputs=model.input,
            outputs=[conv_layer.output, model.output]
        )

        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(img_array)
            if class_index is None:
                class_index = tf.argmax(predictions[0])
            loss = predictions[:, class_index]

        grads = tape.gradient(loss, conv_outputs)
        grads_2 = tf.square(grads)
        grads_3 = grads_2 * grads
        sum_grads = tf.reduce_sum(conv_outputs * grads_2, axis=(1, 2), keepdims=True)
        eps = 1e-10
        alpha = grads_3 / (2 * grads_2 + sum_grads + eps)
        weights = tf.reduce_sum(alpha * tf.nn.relu(grads), axis=(1, 2))
        cam = tf.reduce_sum(weights * conv_outputs, axis=-1)

        heatmap = tf.squeeze(cam)
        heatmap = tf.maximum(heatmap, 0)
        heatmap /= (tf.reduce_max(heatmap) + 1e-10)
        return heatmap.numpy()

    # --- Generate Grad-CAM++ ---
    heatmap = gradcam_plus_plus(model, img_preprocessed, last_conv_layer_name)

    # --- Overlay heatmap ---
    img_orig = cv2.imread(img_path)
    img_orig = cv2.resize(img_orig, (224, 224))
    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img_orig, 0.6, heatmap_color, 0.4, 0)

    # --- Plot side by side ---
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
    plt.title(f"Input Image\nClass: {class_label}")
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title("Grad-CAM++")
    plt.axis('off')
    plt.show()


In [ ]:
model_path = '/content/best_modelVGG16.keras'
model = load_model(model_path)
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

image_path = '/content/NeuroScan/Preprocessed_Training/meningioma_tumor/M_101_DA__7.jpg'
display_gradcam_plus_plus(model, image_path, last_conv_layer_name="block5_conv3", class_names=class_names)

## Lime

In [ ]:
!pip install -q lime

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import numpy as np

def explain_with_lime_side_by_side(img_array, model, predicted_class, class_names=None, num_samples=1000, top_labels=1):
    """
    Show original image and LIME explanation side by side with class name.

    img_array: H x W x C, original image
    model: your fine-tuned VGG16
    predicted_class: int, predicted class index
    class_names: list of class names (optional)
    """
    explainer = lime_image.LimeImageExplainer()

    def model_predict(imgs):
        if imgs.ndim == 3:
            imgs = np.expand_dims(imgs, axis=0)
        from tensorflow.keras.applications.vgg16 import preprocess_input
        imgs_preprocessed = preprocess_input(imgs.astype(np.float32))
        return model.predict(imgs_preprocessed)

    # LIME explanation
    explanation = explainer.explain_instance(
        img_array.astype('double'),
        classifier_fn=model_predict,
        top_labels=top_labels,
        hide_color=0,
        num_samples=num_samples
    )

    temp, mask = explanation.get_image_and_mask(
        label=predicted_class,
        positive_only=False,
        hide_rest=False,
        num_features=10,
        min_weight=0.01
    )

    # Build normalized contribution map
    feature_weights = dict(explanation.local_exp[predicted_class])
    contribution_map = np.zeros(mask.shape, dtype=float)
    for feature, weight in feature_weights.items():
        contribution_map[mask == feature] = weight

    # Normalize to 0-1
    min_val, max_val = contribution_map.min(), contribution_map.max()
    if max_val - min_val > 0:
        contribution_map = (contribution_map - min_val) / (max_val - min_val)
    else:
        contribution_map = np.zeros_like(contribution_map)

    # Plot side by side
    plt.figure(figsize=(12,6))

    # Original image
    plt.subplot(1,2,1)
    plt.imshow(img_array.astype('uint8'))
    plt.title(f"Original Image\nClass: {class_names[predicted_class] if class_names else predicted_class}")
    plt.axis('off')

    # LIME explanation overlay
    plt.subplot(1,2,2)
    plt.imshow(mark_boundaries(temp, mask))
    plt.imshow(contribution_map, cmap='RdYlBu', alpha=0.5)
    plt.title("LIME Explanation")
    plt.colorbar(label='Normalized Contribution (0-1)')
    plt.axis('off')

    plt.show()


In [ ]:
class_names = ["Normal", "Glioma Tumor", "Meningioma Tumor", "Pituitary Tumor"]

image_path = '/content/NeuroScan/Preprocessed_Training/meningioma_tumor/M_497_VF__12.jpg'
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)

predicted_class = np.argmax(model.predict(preprocess_input(np.expand_dims(img_array,0))))

explain_with_lime_side_by_side(img_array, model, predicted_class, class_names=class_names)


# Streamlit app

In [ ]:
!pip install -q streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import cv2

st.title("🧠 Brain Tumor Classification (VGG16)")

model = tf.keras.models.load_model("best_modelVGG16.keras")
IMG_SIZE = 224
CLASSES = ["glioma", "meningioma", "normal", "pituitary"]

uploaded_file = st.file_uploader("Upload MRI image", type=["jpg", "png", "jpeg"])

def preprocess(img):
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = np.expand_dims(img, axis=0)
    return img

if uploaded_file:
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, 1)

    st.image(img, channels="BGR", caption="Uploaded Image")

    input_img = preprocess(img)
    pred = model.predict(input_img)
    class_id = np.argmax(pred)

    st.success(f"Prediction: **{CLASSES[class_id].upper()}**")


In [ ]:
!npm install localtunnel

Open logs and copy ip as password

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com